In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
cle = pd.read_csv('cle_metadata_dnn.csv')
vir = pd.read_csv('vir_metadata_dnn.csv')
hun = pd.read_csv('hun_metadata_dnn.csv')
swi = pd.read_csv('swi_metadata_dnn.csv')

In [3]:
cle_train,cle_test = train_test_split(cle,test_size=0.33, random_state=42)
vir_train,vir_test = train_test_split(vir,test_size=0.33, random_state=42)
hun_train,hun_test = train_test_split(hun,test_size=0.33, random_state=42)
swi_train,swi_test = train_test_split(swi,test_size=0.33, random_state=42)

In [4]:
Train = pd.concat([cle_train,vir_train,hun_train])
Test = pd.concat([cle_test,vir_test,hun_test,swi_test,swi_train])

In [5]:
X_train = Train.iloc[:,:-1]
X_test = Test.iloc[:,:-1]

y_train = Train.iloc[:,-1]
y_test = Test.iloc[:,-1]

Y_train_binary = y_train.apply(lambda x: 1 if x > 0 else 0)
Y_test_binary = y_test.apply(lambda x: 1 if x > 0 else 0)

# CNN

In [6]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define the model architecture
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(64,1)))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit(X_train, Y_train_binary, epochs=1000, batch_size=32,callbacks=[callback])
Y_pred = model.predict(X_test).argmax(axis=1)
    
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred, digits=4))

Epoch 1/1000
17/17 [==============================] - 1s 36ms/step - loss: 0.6970 - accuracy: 0.4897
Epoch 2/1000
17/17 [==============================] - 1s 38ms/step - loss: 0.6929 - accuracy: 0.5253
Epoch 3/1000
17/17 [==============================] - 1s 36ms/step - loss: 0.6929 - accuracy: 0.5178
Epoch 4/1000
17/17 [==============================] - 1s 38ms/step - loss: 0.6933 - accuracy: 0.5178
Epoch 5/1000
17/17 [==============================] - 1s 38ms/step - loss: 0.6930 - accuracy: 0.5178
Epoch 6/1000
17/17 [==============================] - 1s 36ms/step - loss: 0.6937 - accuracy: 0.4916
Epoch 7/1000
17/17 [==============================] - 1s 38ms/step - loss: 0.6933 - accuracy: 0.5178
Epoch 8/1000
13/13 [==============================] - 0s 8ms/step
[[135 252]
 [  0   0]]
              precision    recall  f1-score   support

           0     0.3488    1.0000    0.5172       135
           1     0.0000    0.0000    0.0000       252

    accuracy                         0.3

D:\Anaconda\envs\GPU\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\envs\GPU\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\envs\GPU\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
